# Starting from here

In [1]:
import pandas as pd
import re
from Spacy_Tagging import get_pos_entities_parsed, expandText, expandTextNew
from Wordnet_functionality import get_rootword, isWordPresentInWordnet
# from Task_Extraction_From_LBA_text import get_task_list, is_action_task
import numpy as np
from ast import literal_eval


code = ['A103']
folder_to_save = 'A103/'
training_set = pd.read_csv('Swiss Re-Training_' + code[0] + '.csv')

C:\Users\Karan.Arya\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Creating features for sentence

In [2]:
def getTextFromPOSData(data):
    return ' '.join([d.split('/')[0] for d in data.split(' ')])

In [3]:
len(training_set) * .0005

23.588

In [4]:
threshold_for_vocab_selection = int(len(training_set) * .0005)
print('Selected Threshold: ', threshold_for_vocab_selection)

Selected Threshold:  23


In [5]:
training_set.columns

Index(['Carrier', 'Company Name', 'Coverage Counsel Guidelines',
       'Invoice Date', 'Invoice End Date', 'Invoice Line Item Id',
       'Invoice No.', 'Invoice Start Date', 'Invoice Status',
       'Invoice Status Date', 'Invoice System ID No',
       'Line Item Adjustment Description', 'Line Item Adjustment Reason',
       'Line Item Amount', 'Line Item Client Adjustment',
       'Line Item Date of Service', 'Line Item Description',
       'Line Item ITP Adjustment', 'Line Item Number', 'Line Item Rate',
       'Line Item Reviewer taking Adjustment',
       'Line Item TPA Appeal Adjustment', 'Line Item TPA Net Adjustment',
       'Line Item TPA Reviewer Adjustment', 'Line Item Units',
       'Line Item Vendor Adjustment', 'ML Adj Desc', 'ML Adj Level',
       'ML Adj Reason', 'ML Output 1', 'ML Output 2', 'ML Output 3',
       'ML Score', 'Matter Id', 'Matter No.', 'Received Date', 'Timekeeper',
       'Timekeeper Level', 'UTBMS Activity Code', 'UTBMS Activity Description',
       

In [6]:
# training_set['Line Item Description'].unique().tolist()

In [7]:
stop_word_list = ['void', 'each', 'weigh', 'clip', 'spanish', 'a.m.', 'become', 'semi', 'snow', 'mean', 'cash', 'known', 'many', 'boyfriend', 'example', 'wife', 'ways', '12th']

In [8]:
vocab_dict = {}
for line in training_set[['pos', 'entities']].itertuples():
    pos    = line[1];  pos_modified = pos
    entities = line[2]
    
    if entities!=None and type(entities)!=float:
        if len(entities.strip()) ==0:
            continue
        for entity in entities.split('\t'):
            #print(entity)
            entity_split = entity.split('/')
            entity_text  = entity_split[0]
            tag          = entity_split[1]
            for entity_text_split in entity_text.split(' '):
                if len(entity_text_split.strip())>0:
                    pos_modified = pos_modified.replace(entity_text_split, '<' + tag + '>')
    
    for word in pos_modified.split(' '):
        word_split = word.split('/')
        word_lower = word_split[0].lower()
        if isWordPresentInWordnet(word_lower) or re.search(r'<[A-Za-z]+>', word_lower):
            try:
                word_lower = get_rootword(word_lower, word_split[1])
            except:
                pass
                #print(pos)
                #print()
                #print(entities)
                #print()
                #print(pos_modified)
            #print(word_lower)
            if len(word_lower)>=4 and len(word_split)>1 and word_split[1] in ['VERB', 'ADJ', 'NOUN', 'PROPN']:
                if word_lower in vocab_dict :
                    vocab_dict[word_lower] = vocab_dict[word_lower] + 1
                else:
                    vocab_dict[word_lower] = 1
print(len(vocab_dict))

4463


In [9]:
# vocab_dict

## Bigram and Trigram list

In [10]:
sent_list = []
for line in training_set[['pos', 'entities']].itertuples():
    pos    = line[1];  pos_modified = pos
    entities = line[2]
    
    if entities!=None and type(entities)!=float:
        if len(entities.strip()) ==0:
            continue
        for entity in entities.split('\t'):
            #print(entity)
            entity_split = entity.split('/')
            entity_text  = entity_split[0]
            tag          = entity_split[1]
            for entity_text_split in entity_text.split(' '):
                if len(entity_text_split.strip())>0:
                    pos_modified = pos_modified.replace(entity_text_split, '<' + tag + '>')
                    
                    break
                    
    sent_list.append(re.sub(r'\s+',' ', pos_modified))

In [11]:
len(sent_list)

47176

In [12]:
# for sentence in sent_list:
#     print(sentence)
#     print((re.sub(r'\s+', ' ', getTextFromPOSData(sentence))))
#     print()

In [13]:
from nltk.corpus import stopwords
stop_list = stopwords.words('english')

In [14]:
import string
punctuation_list = []
for char in string.punctuation:
    punctuation_list.append(char)

In [15]:
bigrams_list = []
from nltk import ngrams
for sentence in sent_list:
    clean_sentence = re.sub(r'\s+', ' ', getTextFromPOSData(sentence))

    n = 2
    bigrams = ngrams(clean_sentence.split(), n)
    for grams in bigrams:
        grams = [item.lower() for item in grams]
        bigrams_list.append(grams)

In [16]:
remove_bigram_list = []
for grams in bigrams_list:     
    for word in grams:
        if word in punctuation_list:
            remove_bigram_list.append(grams)
            break
        if word in stop_list:
            remove_bigram_list.append(grams)
            break

In [17]:
remove_bigram_df = pd.DataFrame(remove_bigram_list)
print(remove_bigram_df.shape)
remove_bigram_df = remove_bigram_df.drop_duplicates(subset = [0, 1])
print(remove_bigram_df.shape)

(527161, 2)
(50426, 2)


In [18]:
remove_bigram_array = np.array(remove_bigram_df)

In [19]:
remove_bigram_list = []
for item in remove_bigram_array:
    remove_bigram_list.append(item[0] + '###' + item[1])

In [20]:
from tqdm import tqdm

In [21]:
updated_bigrams_list = []
for item in bigrams_list:
    updated_bigrams_list.append(item[0] + '###' + item[1])

In [22]:
count_bigram_dict = {}
for item in updated_bigrams_list:
    if item not in count_bigram_dict.keys():
        count_bigram_dict[item] = 1
    else:
        count_bigram_dict[item] += 1

In [23]:
import operator
sorted_count_bigram_list = sorted(count_bigram_dict.items(), key=operator.itemgetter(1), reverse = True)

In [24]:
import collections
sorted_count_bigram_dict = dict(collections.OrderedDict(sorted_count_bigram_list))

In [25]:
key_list = list(sorted_count_bigram_dict.keys())

In [26]:
print(len(sorted_count_bigram_dict))
for item in key_list:
    if item in remove_bigram_list:
        del sorted_count_bigram_dict[item]
print(len(sorted_count_bigram_dict))

104273
53847


In [27]:
# sorted_count_bigram_dict

In [28]:
from nltk.corpus import stopwords
stop_list = stopwords.words('english')

import string
punctuation_list = []
for char in string.punctuation:
    punctuation_list.append(char)

In [29]:
trigrams_list = []
from nltk import ngrams
for sentence in sent_list:
    clean_sentence = re.sub(r'\s+', ' ', getTextFromPOSData(sentence))

    n = 3
    trigrams = ngrams(clean_sentence.split(), n)
    for grams in trigrams:
        grams = [item.lower() for item in grams]
        trigrams_list.append(grams)

In [30]:
remove_trigram_list = []
for grams in trigrams_list:   

    for word in grams:
        if word in punctuation_list:
            remove_trigram_list.append(grams)
            break
            
        if (word == grams[0] and word in stop_list) or (word == grams[2] and word in stop_list):
            remove_trigram_list.append(grams)
            break

In [31]:
# remove_trigram_list

In [32]:
remove_trigram_df = pd.DataFrame(remove_trigram_list)
print(remove_trigram_df.shape)
remove_trigram_df = remove_trigram_df.drop_duplicates(subset = [0, 1, 2])
print(remove_trigram_df.shape)

(480186, 3)
(145251, 3)


In [33]:
remove_trigram_array = np.array(remove_trigram_df)

In [34]:
remove_trigram_list = []
for item in remove_trigram_array:
    remove_trigram_list.append(item[0] + '###' + item[1] + '###' + item[2])

In [35]:
from tqdm import tqdm

In [36]:
updated_trigrams_list = []
for item in trigrams_list:
    updated_trigrams_list.append(item[0] + '###' + item[1] + '###' + item[2])

In [37]:
count_trigram_dict = {}
for item in updated_trigrams_list:
    if item not in count_trigram_dict.keys():
        count_trigram_dict[item] = 1
    else:
        count_trigram_dict[item] += 1

In [38]:
import operator
sorted_count_trigram_list = sorted(count_trigram_dict.items(), key=operator.itemgetter(1), reverse = True)

In [39]:
import collections
sorted_count_trigram_dict = dict(collections.OrderedDict(sorted_count_trigram_list))

In [40]:
key_list = list(sorted_count_trigram_dict.keys())

In [41]:
print(len(sorted_count_trigram_dict))
for item in key_list:
    if item in remove_trigram_list:
        del sorted_count_trigram_dict[item]
print(len(sorted_count_trigram_dict))

241542
96291


In [42]:
# fw = open('trigram.txt', 'w', encoding='utf8')
# for word in sorted_count_trigram_dict:
#     fw.write(",".join(word.split('###')) + ","  + str(sorted_count_trigram_dict[word]) + '\n')
# fw.close()

In [43]:
# fw = open('bigram.txt', 'w', encoding='utf8')
# for word in sorted_count_bigram_dict:
#     fw.write(",".join(word.split('###')) + ","  + str(sorted_count_bigram_dict[word]) + '\n')
# fw.close()

In [44]:
import operator
sorted_x = sorted(vocab_dict.items(), key=operator.itemgetter(1))
sorted_x_filtered = [vocab for vocab in sorted_x if vocab[1] >= threshold_for_vocab_selection and vocab not in stop_word_list]
print(len(sorted_x_filtered))

1022


In [45]:
# ### Saving Vocab file
# fw = open('vocab_list_V1.txt', 'w')
# for word in sorted_x_filtered:
#     fw.write(word[0] + '\n')
# fw.close()

In [46]:
## Reading vocab file
vocab_list = set([word for word in open('vocab_list_V1.txt', 'r').read().split('\n') if len(word.strip())>0])

In [47]:
len(vocab_list), len(sorted_count_bigram_dict), len(sorted_count_trigram_dict)

(1022, 53847, 96291)

In [48]:
bigram_dict = {}
for key, value in sorted_count_bigram_dict.items():
    bigram_dict['###'.join([item for item in key.split('###') if len(item) > 3])] = value
print(len(sorted_count_bigram_dict), len(bigram_dict))

53847 50957


In [49]:
trigram_dict = {}
for key, value in sorted_count_trigram_dict.items():
    trigram_dict['###'.join([item for item in key.split('###') if len(key.split('###')[0]) > 3 and len(key.split('###')[2]) > 3])] = value
print(len(sorted_count_trigram_dict), len(trigram_dict))

96291 89204


In [50]:
filtered_bigram_dict = {key: value for key, value in bigram_dict.items() if value > 23}
filtered_trigram_dict = {key: value for key, value in trigram_dict.items() if value > 23}

In [51]:
len(vocab_list), len(filtered_bigram_dict), len(filtered_trigram_dict)

(1022, 1484, 1040)

In [52]:
bigram_vocab_list = list(set(filtered_bigram_dict.keys()))
bigram_vocab_list = set([' '.join(item.split('###')) for item in bigram_vocab_list])

In [53]:
trigram_vocab_list = list(set(filtered_trigram_dict.keys()))
trigram_vocab_list = set([' '.join(item.split('###')) for item in trigram_vocab_list])

In [54]:
len(vocab_list), len(bigram_vocab_list), len(trigram_vocab_list)

(1022, 1484, 1040)

In [ ]:
def generateNGrams(sentence, n):
    ngrams_list = []
    clean_sentence = re.sub(r'\s+', ' ', sentence)
    ngrams_ = ngrams(clean_sentence.split(), n)
    for grams in ngrams_:
        grams = [item.lower() for item in grams]
        ngrams_list.append(' '.join(grams))
        
    return ngrams_list

# generateNGrams('hi i am karAn', 2)

In [ ]:
def creating_feature_file(file_name, data_set):
    fw = open(file_name, 'w', encoding='utf8')
    #fw.write('Sentence,')
    for word in vocab_list:
        fw.write(word + ',')
    for word in bigram_vocab_list:
        fw.write(word + ',')
    for word in trigram_vocab_list:
        fw.write(word + ',')
    fw.write('Line Item Rate,Timekeeper Level,UTBMS Activity Code,UTBMS Phase Code,UTBMS Task Code,Work Area,Work Area Level 2')
    fw.write(',Is_Prior_Approved,Is_Action,Class_Label\n')

    for line in data_set[['pos', 'Line Item Rate','Timekeeper Level','UTBMS Activity Code',\
             'Work Area', 'Work Area Level 2','Is_Prior_Approved', \
             'Is_Action', 'Adjusted', 'UTBMS Phase Code',  'UTBMS Task Code', 'Line Item Description']].itertuples():
        sentence_root = [get_rootword(word.split('/')[0], word.split('/')[1]) for word in line[1].split()]
        intersection_list = list(set(sentence_root) & vocab_list)
        #fw.write(line.sentence_text + ',')
        for word in vocab_list:
            if word in intersection_list:
                fw.write(str(1) + ',')
            else:
                fw.write(str(0) + ',')
                
        ## Bigram and Trigram logic
        bigram_sentence_root = generateNGrams(line[11], n = 2)
        bigram_intersection_list = list(set(bigram_sentence_root) & bigram_vocab_list)
        for word in bigram_vocab_list:
            if word in bigram_intersection_list:
                fw.write(str(1) + ',')
            else:
                fw.write(str(0) + ',')
                
        trigram_sentence_root = generateNGrams(line[11], n = 3)
        trigram_intersection_list = list(set(trigram_sentence_root) & trigram_vocab_list)
        for word in trigram_vocab_list:
            if word in trigram_intersection_list:
                fw.write(str(1) + ',')
            else:
                fw.write(str(0) + ',')
        
        ## Line Item Rate, Timekeeper Level,UTBMS Activity Code'
        fw.write(str(line[2]) + "," + str(line[3]) + "," + str(line[4]))
        ## 'UTBMS Phase Code', 'UTBMS Task Code',
        fw.write("," + str(line[10]) + "," + str(line[11]))
        ## 'Work Area', 'Work Area Level 2',
        fw.write("," + line[5] + "," + line[6])
        # 'Is_Prior_Approved','Is_Action'
        if line[7]:
            fw.write("," + str(1) + "," + str(line[8]))
        else:
            fw.write("," + str(0) + "," + str(line[8]))
        
        ## Adjusted
        fw.write("," + str(line[9]) + '\n')

    fw.close()

In [ ]:
creating_feature_file('bag_of_words_training_V1.csv', training_set)

In [ ]:
df = pd.read_excel('bag_of_words_training_V1.csv',
                        sep='|',  dtype='unicode', error_bad_lines=False)
# except CParserError:
#     print("Something wrong the file")

In [ ]:
len(df.columns)